In [ ]:
import json
import time
from collections import defaultdict
from pathlib import Path


import cv2
import numpy as np
import pandas as pd

from ultralytics import YOLO

In [ ]:

zones = []
current_zone = []
drawing = False

# Resize helper
def resize_frame(frame, max_width=1280, max_height=720):
    h, w = frame.shape[:2]
    scale = min(max_width / w, max_height / h)
    new_w, new_h = int(w * scale), int(h * scale)
    return cv2.resize(frame, (new_w, new_h)), scale

def mouse_callback(event, x, y, flags, param):
    global current_zone, drawing
    if event == cv2.EVENT_LBUTTONDOWN:   # Left click = add point
        current_zone.append((x, y))
        drawing = True
    elif event == cv2.EVENT_RBUTTONDOWN:  # Right click = finish polygon
        if len(current_zone) >= 3:
            zones.append({"points": current_zone.copy()})
            print(f"Zone saved: {current_zone}")
        current_zone = []
        drawing = False

video_path = r"D:\Project\Traffic\traffic.mp4"
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
cap.release()

# Resize frame to fit screen
frame, scale = resize_frame(frame, max_width=1280, max_height=720)

cv2.namedWindow("Draw Zones (Left=Add, Right=Finish, ESC=Exit)")
cv2.setMouseCallback("Draw Zones (Left=Add, Right=Finish, ESC=Exit)", mouse_callback)

while True:
    temp_frame = frame.copy()

    # Draw finished zones
    for z in zones:
        pts = [(int(p[0]), int(p[1])) for p in z["points"]]
        cv2.polylines(temp_frame, [np.array(pts, np.int32)], True, (0,255,0), 2)

    # Draw current in-progress zone
    if current_zone:
        pts = [(int(p[0]), int(p[1])) for p in current_zone]
        cv2.polylines(temp_frame, [np.array(pts, np.int32)], False, (0,0,255), 2)

    cv2.imshow("Draw Zones (Left=Add, Right=Finish, ESC=Exit)", temp_frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC = exit
        break

cv2.destroyAllWindows()

# Save zones to JSON
with open("zones.json", "w") as f:
    json.dump({"zones": zones}, f, indent=2)

print("Zones saved to zones.json ✅")


In [41]:
def load_zones(zones_file="zones.json"):
    with open(zones_file, "r") as f:
        data = json.load(f)
    zones = []
    for zone in data["zones"]:
        poly = np.array(zone["points"], dtype=np.int32)
        zones.append(poly)
    return zones

# ---- Helper: Point in Polygon ----
def is_point_in_poly(point, poly):
    # Ensure point is tuple of Python floats/ints
    pt = (float(point[0]), float(point[1]))
    return cv2.pointPolygonTest(poly, pt, False) >= 0


In [ ]:
def resize_frame(frame, max_width=1280):
    """Resize frame keeping aspect ratio and return only the image (not a tuple)."""
    h, w = frame.shape[:2]
    if w > max_width:
        ratio = max_width / w
        frame = cv2.resize(frame, (int(w * ratio), int(h * ratio)))
    return frame  # <-- return only the NumPy array

CLASS_COLORS = {
    2: (0, 255, 0),    # Green for car
    5: (255, 0, 0),    # Blue for bus
    7: (0, 0, 255)     # Red for truck
}
CLASS_NAMES = {
    2: "car",
    5: "bus",
    7: "truck"
}

import cv2
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from ultralytics import YOLO  # or your YOLO import

# -----------------------------
# Your resize_frame function remains unchanged
# -----------------------------

CLASS_COLORS = {
    2: (0, 255, 0),    # Green for car
    5: (255, 0, 0),    # Blue for bus
    7: (0, 0, 255)     # Red for truck
}
CLASS_NAMES = {
    2: "car",
    5: "bus",
    7: "truck"
}
def process_video(video_path, model_path, output_path, save_csv, zones_file, resize_width=1280):
    model = YOLO(model_path)
    zones = load_zones(zones_file) if zones_file else []

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {video_path}")

    ret, frame = cap.read()
    if not ret:
        raise IOError("Cannot read first frame")
    frame = resize_frame(frame, max_width=resize_width)
    height, width = frame.shape[:2]
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Global unique vehicle IDs per zone per class
    unique_ids_global = defaultdict(set)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = resize_frame(frame, max_width=resize_width)
        if frame.dtype != np.uint8:
            frame = frame.astype(np.uint8)

        results = model.track(source=frame, persist=True, tracker="bytetrack.yaml")
        res = next(iter(results))

        boxes, scores, track_ids, classes = [], [], [], []

        for idx, box in enumerate(res.boxes.xyxy):
            cls = int(res.boxes.cls[idx])
            if cls not in [2,5,7]:
                continue
            track_id = res.boxes.id[idx]
            conf = float(res.boxes.conf[idx])
            x1, y1, x2, y2 = map(int, box)
            w, h = x2 - x1, y2 - y1
            boxes.append([x1, y1, w, h])
            scores.append(conf)
            track_ids.append(track_id)
            classes.append(cls)

        if boxes:
            indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.3, nms_threshold=0.5)
            for i in indices.flatten():
                x, y, w, h = boxes[i]
                cx, cy = x + w//2, y + h//2
                track_id = track_ids[i]
                cls_id = classes[i]

                # Count unique if inside zone
                for j, poly in enumerate(zones):
                    key = f"zone_{j}_class_{cls_id}"
                    if is_point_in_poly((cx, cy), poly):
                        unique_ids_global[key].add(track_id)

                # Draw box
                color = CLASS_COLORS[cls_id]
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.circle(frame, (cx, cy), 3, (0,0,0), -1)

        # Draw zones and cumulative counts
        for j, poly in enumerate(zones):
            cv2.polylines(frame, [poly], True, (255,255,0), 2)
            y_offset = 0
            for cls_id in [2,5,7]:
                key = f"zone_{j}_class_{cls_id}"
                text = f"{CLASS_NAMES[cls_id]}: {len(unique_ids_global.get(key,set()))}"
                cv2.putText(frame, text, (poly[0][0], poly[0][1]-30-y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, CLASS_COLORS[cls_id], 2)
                y_offset += 25

        out.write(frame)
        cv2.imshow("Traffic Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Prepare final counts for CSV
    final_counts = {}
    for j, poly in enumerate(zones):
        for cls_id in [2,5,7]:
            key = f"zone_{j}_class_{cls_id}"
            final_counts[key] = len(unique_ids_global.get(key, set()))

    df = pd.DataFrame([final_counts])
    if save_csv:
        df.to_csv(save_csv, index=False)

    return df

if __name__ == "__main__":
    video_path = r"D:\Project\Traffic\traffic.mp4"
    df = process_video(
        video_path,
        model_path="yolo11n.pt",
        output_path="traffic_out.mp4",
        save_csv="traffic_counts.csv",
        zones_file="zones.json",
        resize_width=1280
    )
    print(df)


0: 384x640 1 person, 12 cars, 1 bus, 4 trucks, 8.0ms
Speed: 15.4ms preprocess, 8.0ms inference, 12.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 12 cars, 1 bus, 4 trucks, 7.6ms
Speed: 1.1ms preprocess, 7.6ms inference, 16.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 3 trucks, 8.0ms
Speed: 1.1ms preprocess, 8.0ms inference, 10.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 2 trucks, 7.9ms
Speed: 1.0ms preprocess, 7.9ms inference, 16.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 2 trucks, 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 16.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 2 trucks, 7.5ms
Speed: 1.1ms preprocess, 7.5ms inference, 13.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 3 trucks, 7.5ms
Speed: 1.1ms preprocess, 7.5ms inference, 14.2ms postprocess per image at shape (1, 3, 384, 640)

0: 38